In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score, precision_score
from scipy.stats import norm

# Sử dụng thư viện numpy và pandas

Đọc dữ liệu từ file, xử lý dữ liệu và phân chia dữ liệu

In [16]:
df = pd.read_csv('/home/vitquay1708/Study_Space/ml/week_1/data/breast-cancer-wisconsin.data', header=None)
df = df.drop(0, axis = 1) # Bỏ cột 0 là mã số mẫu vì không liên quan tới bài toán
#Ép các cột (trừ cột label về số)
for col in df.columns[:-1]:
  df[col] = pd.to_numeric(df[col], errors="coerce")

df = df.fillna(df.mean()) # Ghi đè các giá trị Nan bằng trung bình với giả thiết các trường dữ liệu được phân phối theo phân bố chuẩn
benign = df[df[10] == 2]
malignant = df[df[10] == 4]
#Lấy 80 dòng ngẫu nhiên của lành tính và 40 dòng ngẫu nhiên của ác tính
benign_test = benign.sample(n = 80, random_state=17)
malignant_test = malignant.sample(n = 40, random_state=17)

#Lấy phần còn lại của lành tính và ác tính làm tập train
benign_train = benign.drop(benign_test.index)
malignant_train = malignant.drop(malignant_test.index)
# Gộp train và test vào làm X_train, X_test, y_train, y_test
X_train = pd.concat([benign_train.iloc[:, :9], malignant_train.iloc[:, :9]], axis=0)
y_train = pd.concat([benign_train.iloc[:, 9], malignant_train.iloc[:, 9]], axis=0)
X_test = pd.concat([benign_test.iloc[:, :9], malignant_test.iloc[:, :9]], axis=0)
y_test = pd.concat([benign_test.iloc[:, 9], malignant_test.iloc[:, 9]], axis=0)



print(len(X_train), len(X_test), len(y_train), len(y_test))
# print(X_train)
print(df.dtypes)

579 120 579 120
1       int64
2       int64
3       int64
4       int64
5       int64
6     float64
7       int64
8       int64
9       int64
10      int64
dtype: object


Hàm dự đoán Naive Bayes

In [17]:
def predict_NB_gaussian_class(X,mu_list,std_list,pi_list):
#Returns the class for which the Gaussian Naive Bayes objective function has greatest value
    scores_list = []
    classes = len(mu_list)
    for p in range(classes):
        score = (norm.pdf(x = X[0], loc = mu_list[p][0][0], scale = std_list[p][0][0])
        * norm.pdf(x = X[1], loc = mu_list[p][0][1], scale = std_list[p][0][1])
        * norm.pdf(x = X[2], loc = mu_list[p][0][2], scale = std_list[p][0][2])
        * norm.pdf(x = X[3], loc = mu_list[p][0][3], scale = std_list[p][0][3] )
        * norm.pdf(x = X[4], loc = mu_list[p][0][4], scale = std_list[p][0][4])
        * norm.pdf(x = X[5], loc = mu_list[p][0][5], scale = std_list[p][0][5])
        * norm.pdf(x = X[6], loc = mu_list[p][0][6], scale = std_list[p][0][6])
        * norm.pdf(x = X[7], loc = mu_list[p][0][7], scale = std_list[p][0][7])
        * norm.pdf(x = X[8], loc = mu_list[p][0][8], scale = std_list[p][0][8] )
        * pi_list[p])
        scores_list.append(score)
        
    return 2 if np.argmax(scores_list) == 0 else 4

Tính giá trị trung bình, độ lệch chuẩn, xác suất theo từng lớp của dữ liệu

In [18]:
mu_list = np.split(df.groupby(9).mean().values, [1])
std_list = np.split(df.groupby(9).std().values, [1], axis=0)
pi_list = df.iloc[:, 9].value_counts().values / len(df)
print(mu_list)
print(std_list)
print(pi_list)

[array([[3.85146805, 2.44905009, 2.57858377, 2.2193437 , 2.75474957,
        2.78879878, 3.00345423, 2.18480138, 2.46286701]]), array([[ 7.08571429,  4.88571429,  4.8       ,  4.14285714,  3.8       ,
         6.91428571,  4.77142857,  5.54285714,  3.54285714],
       [ 7.21212121,  6.6969697 ,  6.48484848,  5.60606061,  5.81818182,
         7.33333333,  5.63636364,  5.81818182,  3.87878788],
       [ 7.16666667,  6.66666667,  5.66666667,  7.5       ,  5.5       ,
         7.91666667,  5.33333333,  7.33333333,  4.        ],
       [ 6.33333333,  6.83333333,  7.        ,  5.66666667,  5.        ,
         7.33333333,  5.83333333,  4.        ,  3.66666667],
       [ 8.        ,  8.33333333,  7.66666667,  5.        ,  5.33333333,
        10.        ,  7.33333333,  8.        ,  4.        ],
       [ 6.        ,  6.88888889,  7.11111111,  7.33333333,  6.66666667,
         7.55555556,  6.22222222,  6.88888889,  3.77777778],
       [ 6.75      ,  6.5       ,  7.125     ,  5.375     ,  5.25   

Thực hiện tính toán phân loại trên tập train và tập test

Tập train:

In [19]:
y_train_pred = np.zeros(X_train.shape[0])
for i in range(X_train.shape[0]):
    x = X_train.iloc[i].values
    y_train_pred[i] = predict_NB_gaussian_class(x, mu_list, std_list, pi_list)
print('Accuracy (train): ', accuracy_score(y_train, y_train_pred))
print('Precision (train): ', precision_score(y_train, y_train_pred, pos_label=4))
print('Recall (train): ', recall_score(y_train, y_train_pred, pos_label=4))

Accuracy (train):  0.9309153713298791
Precision (train):  0.9707602339181286
Recall (train):  0.8258706467661692


Tập test

In [20]:
y_test_pred = np.zeros(X_test.shape[0])
# print(X_train.iloc[0].values.reshape(-1, 1))
for i in range(X_test.shape[0]):
    x = X_test.iloc[i].values
    y_test_pred[i] = predict_NB_gaussian_class(x, mu_list, std_list, pi_list)
print('Accuracy (test): ', accuracy_score(y_test, y_test_pred))
print('Precision (test): ', precision_score(y_test, y_test_pred, pos_label=4))
print('Recall (test): ', recall_score(y_test, y_test_pred, pos_label=4))

Accuracy (test):  0.9666666666666667
Precision (test):  1.0
Recall (test):  0.9


# Thư viện sklearn

In [21]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)


sklearn_y_test_pred = model.predict(X_test)

print(accuracy_score(sklearn_y_test_pred, y_test))
print(precision_score(sklearn_y_test_pred, y_test, pos_label=4))
print(recall_score(sklearn_y_test_pred, y_test, pos_label=4))

0.9916666666666667
1.0
0.975609756097561
